<a href="https://colab.research.google.com/github/davetew/Zero-Carbon-Aviation/blob/master/Dual_Cycle_Propulsion_System_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install any non-default colab setup packages
!pip install pint

     |████████████████████████████████| 209 kB 30.6 MB/s 


In [ ]:
#@title Importing the required colab-standard Python modules
import numpy as np

import pint
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE, titleweight="bold")     # font size & weight of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE, titleweight="bold")    # font size & weight of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE, titleweight="bold") 

import seaborn as sns
import pandas as pd
import datetime

from copy import deepcopy

from scipy.optimize import brentq, minimize, Bounds

In [57]:
# Compressible flow relations

def Tt_T(Mach, γ=1.4):
  """Total to static temperature ratio"""
  return 1 + (γ - 1)/2 * Mach**2

def Tstatic(Tt, Mach, γ=1.4):
  """Static temperature"""
  return Tt / ( 1 + (γ-1)/2*Mach**2)

def Pt_p(Mach, γ=1.4):
  """Total to static pressure ratio"""
  return Tt_T(Mach, γ)**(γ/(γ-1))

def calcMach(Tt_t, γ=1.4):
  """Mach number"""
  return np.sqrt( 2/(γ-1)*(Tt_t - 1 ) ) 

# Turbomachinery exit conditions
def compExitT(InletT, PR, ηpoly, γ=1.4):
  """Calculate the compressor exit temperature given 
    1. the inlet temperature, 
    2. the pressure ratio,
    3. the polytropic efficiency, and
    4. the ratio of specific heats"""
  return InletT*PR**((γ-1)/γ/ηpoly)


## Simple Turbine Fan
`SimpleTurboFan` is intended to facilitate estimates of the overall efficiency and massflow-specific thrust of turbofan engines given the required design and operating parameters.

The design parameters include--
1. $BypRatio$: the bypass to core mass flow ratio
2. $OPR$: the overall pressure ratio--compressor exit total to fan inlet total,
4. $FPR$: the fan pressure ratio--fan exit total to fan inlet total,
5. $η^{poly}_{fan}$: the fan polytropic efficiency,
6. $η^{poly}_{compressor}$: the compressor polytropic efficiency,
7. $η^{poly}_{turbine}$: the turbine polytropic efficiency,
8. $T^{metal}_{turbine}$: the turbine metal temperature,
9. $St_{turbine}$: the Stanton number for turbine cooling heat transfer,
10. $PR_{inlet}$: the inlet total pressure ratio,
11. $PR_{combustor}$: the combustor total pressure ratio,
12. $PR_{nozzle}$: the core/primary nozzle inflow total to exit static pressure ratio

The operating parameters include--
1. $M_{flight}$: the flight Mach number 
2. $p_{ambient}$: the ambient static pressure
3. $T_{ambient}$: the ambient static temperature
4. $R_{ambient}$: the ambient gas-specific gas constant in $\frac{J}{kgK}$
5. $\gamma$: the ambient ratio of specfic heats

### Thrust Calculation

The net thrust of the propulsion system is given by

$$T = \dot{m}_{bypass} U^{exit}_{fan} + \dot{m}_{core}U^{exit}_{core} - \left( \dot{m}_{bypass} + \dot{m}_{core} \right) U_{\infty} $$



The net thrust non-dimensionalized by the "core inflow" momentum flux is given by

\begin{align}
\frac{T}{\dot{m}_{core}U_{\infty}} & = \beta \frac{U^{exit}_{fan}}{U_{\infty}} + \frac{U^{exit}_{core}}{U_{\infty}} - \left( 1 + \beta \right) \\ 
& = \beta \left( \frac{U^{exit}_{fan}}{U_{\infty}} - 1 \right) + \frac{U^{exit}_{core}}{U_{\infty}} -  1 \\ 
& = T''
\end{align}

The fan exit to flight velocity ratio may be expressed as

$$ \frac{U^{exit}_{fan}}{U_{\infty}} = \frac{M^{exit}_{fan}}{M_{\infty}} \sqrt{\frac{T^{exit}_{fan}}{T_{\infty}}}$$

The fan exit Mach number may be calculated from

$$M^{exit}_{fan} = \sqrt{ \frac{2}{\gamma-1}\left[\left(\frac{Pt^{exit}_{fan}}{p_{\infty}}\right)^{\frac{\gamma-1}{\gamma}} - 1 \right]}$$

The fan exit static temperature may be calculated from

$$T^{exit}_{fan} = \frac{Tt^{exit}_{fan}}{1 + \frac{\gamma-1}{2}\left(M^{exit}_{fan}\right)^2}$$

The core exit to flight velocity ratio may likewise be expressed 

$$ \frac{U^{exit}_{core}}{U_{\infty}} = \frac{M^{exit}_{core}}{M_{\infty}} \sqrt{\frac{T^{exit}_{core}}{T_{\infty}}}$$

### Overall Efficiency Calculation

The overall efficiency of the propulsion system is define as follows

$$\eta_{overall} = \frac{TU_{\infty}}{\dot{Q}_{fuel}}$$

$\dot{Q}_{fuel}$ is the thermal energy released via the combustion of the fuel.

\begin{align}
\eta_{overall} & = \frac{T}{\dot{m}_{core}U_{\infty}} \frac{\dot{m}_{core}U^{2}_{\infty}}{\dot{Q}_{fuel}} \\
& = \frac{T''U_{\infty}^2}{\dot{q}_{fuel}} 
\end{align}


In [66]:
class SimpleTurboFan:

  def __init__(self, InletMach=0.8, BypRatio=None, 
               PR = {'inlet': 1.0, 'fan': 1.4, 'overall': 20, 
                     'burner': 0.98, 'cooling': 0.95, 'nozzle': 0.8},
               ηpoly = {'fan': 1.0, 'compressor': 1.0, 'turbine': 1.0},
               ambient = {'T': Q_(25, 'degC').to('K'), 'p': Q_(100, 'kPa'), 'R': Q_(287.058, 'J/kg/K'), 'γ': 1.4},
               turbine = {'inletT': Q_(1200, 'degC').to('K'), 'metalT': Q_(1200, 'degC').to('K'), 'StantonNum': 0.07}):
    """Define an instance of SimpleTurbonFan given
    1. InletMach: the fan inlet/flight Mach number,
    2. BypRatio: the fan to core mass flow ratio,
    3. OPR: the overall pressure ratio--compressor exit total to fan inlet total,
    4. FRP: the fan pressure ratio--fan exit total to fan inlet total,
    5. ηpoly_fan: the fan polytropic efficiency,
    6. ηpoly_compressor: the compressor polytropic efficiency,
    7. ηpoly_turbine: the turbine polytropic efficiency,
    8. T_ambient_C: the ambient static temperature in deg C,
    9. p_ambient_kPa: the ambient static pressure in kPa,
    10. R_J_kgK: the ambient/inflow gas-specific gas constant in J/kg/K,
    11. γ: the ambient/inflow ratio of specific heats,
    13. Turbine_Metal_Temperature_C: ,
    14. StantonNumber:
    15. inletPR: the inlet total pressure ratio,
    16: burnerPR: the burner/combustor total pressure ratio,
    17: coolingPR: the total pressure ratio across the cooling system--between the compressor exit and the turbine blades,
    18: nozzlePR: the core exhaust nozzle total to static pressure ratio.
    """  
    
    self.InletMach = InletMach
    self.PR = PR
    self.ηpoly = ηpoly
    
    self.ambient = ambient
    self.ambient['cp'] =  self.ambient['R'] * self.ambient['γ'] / (self.ambient['γ']-1)

    self.turbine = turbine

    if self.turbine['metalT'] is None:
      self.turbine['metalT'] = turbine['inletT']

    self.T_combustor_exit = turbine['inletT'] 

    if BypRatio is None and PR['fan'] is None:
      raise ValueError('Either BypRatio or FPR must be specified.')
    elif BypRatio is None:
      # Calculate the bypass ratio that the core is capable of enabling
      self.BypRatio = self.calcBypRatio()
    elif PR['fan'] is None:
      # Calculate the fan pressure ratio that the core is capable of enabling
      self.PR['fan'] = None 
    else:
      raise ValueError('Specify either BypRatio or FPR.')

  def calcBypRatio(self):
    """Calculate the bypass ratio that results in no net fan-compressor-turbine shaft work"""

    def δwork(BypRatio):
      return SimpleTurboFan(InletMach=self.InletMach,
                            BypRatio=BypRatio,
                            PR=self.PR,
                            ηpoly = self.ηpoly,
                            ambient = self.ambient,
                            turbine = self.turbine).mass_specific_work

    # Calculate the PR that results in the target mass specific work
    try:
      bypratip, results = brentq(δwork, 1, 40, full_output=True)
    except ValueError:
      return np.nan

    # Return an answer if the solver converges
    return bypratio if results.converged else np.nan

  def optimize(self):
    """Optimize the cycle for maximum efficiency subject to the component efficiency and material
    temperature limits provided"""

    def inefficiency(x):
      # Copy the input params into the appropriate dictionary for initiation of an instance of SimpleTurboFan
      # x = [Fan PR, Compressor PR, TIT, Bypass Ratio]
      PR = deepcopy(self.PR) 
      PR['fan'] = x[0]; PR['compressor'] = x[1]
      turbine = deepcopy(self.turbine)
      turbine['inletT'] = x[2]

      """Calculate and return the 'inefficiency' or 1 - efficiency"""
      return 1 - SimpleTurboFan(InletMach=self.InletMach,
                                BypRatio=x[3],
                                PR=PR,
                                ηpoly = self.ηpoly,
                                ambient = self.ambient,
                                turbine = turbine).overall_efficiency

    try:
      results = minimize(inefficiency, 
                         [self.PR['fan'], self.PR['compressor'], turbine['inletT'], self.BypRatio], 
                         method='trust-constr', 
                         bounds=Bounds([1, 10], [1, 100], 
                                       [self.ambient['T'].to('K').magnitude, 2273]
                                       [0, 20]))

    except ValueError:
      return None
    
    else:
      if results.status == 1 or results.status == 2:
        self.PR['fan'] = results.x[0]
        self.PR['compressor'] = results.x[1]
        self.T_combustor_exit = Q_(results.x[2], 'K')
        self.BypRatio = results.x[3]
        return self 
      else:
        return None      

  def sonicVel(self, T, γ=None, R=None):
    """Speed of sound in m/s"""

    # Ambient conditions are default
    γ = self.ambient['γ'] if γ is None else γ
    R = self.ambient['R'] if R is None else R

    return np.sqrt(γ*R*T).to('m/s')

  @property
  def U_flight(self):
    """Flight velocity in m/s"""
    return self.InletMach*self.sonicVel(self.ambient["T"]).to('m/s')

  @property
  def T_inlet(self):
    """Inlet total temperature in K"""
    return ( self.ambient['T']*Tt_T(self.InletMach, self.ambient['γ'])).to('K')

  @property
  def P_inlet(self):
    """Inlet total pressure in Pa"""
    return ( self.ambient['p']*Pt_p(self.InletMach, self.ambient['γ'])).to('Pa')

  @property
  def T_fan_exit(self):
    """Fan exit total temperature in K"""
    return compExitT(self.T_inlet, self.PR['fan'], self.ηpoly["fan"], self.ambient["γ"]).to('K')

  @property
  def P_fan_exit(self):
    """Fan exit total pressure in Pa"""
    return ( self.P_inlet*self.PR['fan'] ).to('Pa')
    
  @property
  def T_compressor_exit(self):
    """Compressor exit total temperature in K"""
    return compExitT(self.T_fan_exit, self.PR['overall']/self.PR['fan'], self.ηpoly["compressor"], self.ambient["γ"]).to('K')

  @property
  def P_compressor_exit(self):
    """Compressor exit total pressure in Pa"""
    return ( self.P_inlet*self.PR['overall'] ).to('Pa')

  @property
  def P_combustor_exit(self):
    """Combustor exit total pressure in Pa"""
    return (self.P_compressor_exit*self.PR['burner']).to('Pa')

  @property
  def βcooling(self):
    """Turbine cooling / inlet mass flow ratio"""
    return ( self.turbine['StantonNum']*(self.T_combustor_exit-self.turbine['metalT']) /
            (self.turbine['metalT'] - self.T_compressor_exit) )
    
  @property
  def P_turbine_exit(self):
    """Turbine exit total pressure in Pa"""
    return self.PR['nozzle']*self.ambient['p'].to('Pa')

  @property
  def turbinePR(self):
    """Turbine inlet to exhaust total pressure ratio"""
    return ( self.P_combustor_exit*(1-self.βcooling) + self.P_compressor_exit*self.βcooling ) / self.P_turbine_exit

  @property
  def T_turbine_inlet(self):
    """Temperature after mixing of combustor exit and cooling flows in K"""
    return ( self.T_compressor_exit*self.βcooling + self.T_combustor_exit*(1-self.βcooling) ).to('K')
  
  @property
  def T_turbine_exit(self):
    """Turbine exit temperature in K"""
    return ( self.T_turbine_inlet*self.turbinePR**(-(self.ambient["γ"]-1)*self.ηpoly["turbine"]/self.ambient["γ"]) ).to('K')

  @property
  def mass_specific_heat_addition(self):
    """Core mass specific heat addition in J/kg"""
    return ( (self.T_combustor_exit - self.T_compressor_exit)*self.ambient["cp"]*(1-self.βcooling) ).to('J/kg')

  @property
  def fan_core_mass_specific_work(self):
    """Calculate the fan core-mass-specific input work in J/kg of core flow"""
    return ( self.ambient["cp"]*(1+self.BypRatio)*(self.T_fan_exit - self.T_inlet) ).to('J/kg')

  @property 
  def compressor_specific_work(self):
    """Calculate the compessor mass-specific input work in J/kg of core flow"""
    return ( self.ambient["cp"]*(self.T_compessor_exit - self.T_fan_exit) ).to('J/kg')

  @property
  def turbine_specific_work(self):
    """Calculate the turbine mass-specific output work in J/kg of core flow"""
    return ( self.ambient["cp"]*(self.T_turbine_inlet - self.T_turbine_exit) ).to('J/kg')

  @property
  def mass_specific_work(self):
    """Net mass specific work in J/kg of core flow"""
    return ( self.turbine_specific_work - self.compressor_specific_work -
            self.fan_core_mass_specific_work ).to('J/kg')
    
  @property
  def M_fan_exit(self):
    """Fan exit Mach number"""
    return calcMach( (self.P_fan_exit/self.ambient["p_Pa"])**(1/self.ambient["γ1"]), self.ambient["γ"] )

  @property
  def U_fan_exit(self):
    """Fan exit velocity"""
    return self.M_fan_exit*self.sonicVel(Tstatic(self.T_fan_exit, self.M_fan_exit, self.ambient["γ"]))

  @property
  def M_core_exit(self):
    """Core nozzle exit Mach number"""
    return calcMach( (self.P_turbine_exit/self.ambient["p_Pa"])**(1/self.ambient["γ1"]), self.ambient["γ"] )

  @property
  def U_core_exit(self):
    """Core exit velocity in m/s"""
    return ( self.M_core_exit*self.sonicVel(Tstatic(self.T_turbine_exit, self.M_core_exit, self.ambient["γ"])) ).to('m/s')

  @property
  def thrust(self):
    """Thrust non-dimensionalized by the "core inflow" momentum flux"""
    return self.BypRatio*( self.U_fan_exit/self.U_flight - 1 ) + self.U_core_exit / self.U_flight - 1 

  @property
  def thermal_efficiency(self):
    """Core thermal efficiency"""
    return ( self.turbine_specific_work - self.compressor_specific_work ) / self.mass_specific_heat_addition

  @property
  def overall_efficiency(self):
    """Overall propulsion system efficiency"""
    return self.thrust*self.U_flight**2/self.mass_specific_heat_addition

  @property
  def propulsive_efficiency(self):
    """Propulsive efficiency"""
    return self.overall_efficiency / self.thermal_efficiency

  @property
  def cycleTemperatures(self):
    """Cycle total temperatures"""
    return np.array([self.T_inlet, self.T_fan_exit, self.T_compressor_exit,
                    self.T_combustor_exit, self.T_turbine_inlet, self.T_turbine_exit])

  @property
  def cyclePressures(self):
    """Cycle total pressures"""
    return np.array([self.P_inlet, self.P_fan_exit, self.P_compressor_exit,
                     self.P_combustor_exit, self.P_combustor_exit, self.P_turbine_exit])

  @property
  def cycleEnthalpies(self):
    return np.array([self.enthalpy(T) for T in self.cycleTemperatures])

  @property
  def cycleEntropies(self):
    return np.array( [self.entropy(T, p) for T, p in 
                      zip(self.cycleTemperatures, self.cyclePressures)] )  
    
  def enthalpy(self, T):
    """Calcuate and return the mass-specific enthalpy in J/kg"""
    return ( self.ambient["cp"]*(T - self.ambient["T"]) ).to('J/kg')

  def entropy(self, T, p):
    """Calcuate and return the mass-specific entropy in J/kg/K"""
    return ( self.ambient["cp"]*np.log(T/self.ambient["T"]) -
            self.ambient["R"]*np.log(p/self.ambient["p"]) ).to('J/kg/K')

  def cycleDiagrams(self):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,6))

    # Temperature-Entropy Diagram
    ax1.plot(self.cycleEntropies, self.cycleTemperatures, marker='d')
    ax1.set_xlabel('Entropy (J/kg/K)')
    ax1.set_ylabel('Temperature ($^{\circ}$C)')
    ax1.grid()
    ax1.set_title(f'TS: Efficiency= {self.efficiency*100:0.0f}%')
    
    # Pressure-Enthalpy
    ax2.plot(self.cycleEnthalpies/1000, self.cyclePressures/1000, marker='p')
    ax2.set_xlabel('Enthalpy (kJ/kg)')
    ax2.set_ylabel('Pressure (kPa)')
    ax2.grid()
    ax2.set_title(f'PH: Specific Work={self.mass_specific_work/1000:.0f} kJ/kg')

In [67]:
test = SimpleTurboFan().cycleTemperatures
print(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:260: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.


ValueError: ignored